# Simulating lightcurves for SNe IIP

In [4]:
import os
home_dir = os.environ.get('HOME')

# Please enter the filename of the ztf_sim output file you would like to use. The example first determines
# your home directory and then uses a relative path (useful if working on several machines with different usernames)
survey_file = os.path.join(home_dir, 'data/ZTF/one_year_sim_incomplete.db')

# Please enter the path to where you have placed the Schlegel, Finkbeiner & Davis (1998) dust map files
# You can also set the environment variable SFD_DIR to this path (in that case the variable below should be None)
sfd98_dir = os.path.join(home_dir, 'data/sfd98')

In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from astropy.cosmology import Planck15
import simsurvey
import sncosmo

In [6]:
# Load the CCD corners from file
ccd_corners = np.genfromtxt('data/ZTF_corners.txt')
ccds = [ccd_corners[4*k+16:4*k+20] for k in range(16)]

In [7]:
# Load simulated survey from file (download from ftp://ftp.astro.caltech.edu/users/ebellm/one_year_sim_incomplete.db)
# Currently DES filters are used as proxies for ZTF filters
plan = simsurvey.SurveyPlan(load_opsim=survey_file, band_dict={'g': 'desg', 'r': 'desr'}, ccds=ccds)

mjd_range = (plan.cadence['time'].min(), plan.cadence['time'].max())

In [ ]:
dust = sncosmo.CCM89Dust()
model = sncosmo.Model(source='s11-2005lc',
                      effects=[dust],
                      effect_names=['host'],
                      effect_frames=['rest'])

def random_parameters(redshifts, model,
                      mag=(-19.3, 0.1),
                      r_v=2., ebv_rate=0.11,
                      **kwargs):
    # Amplitude
    amp = []
    for z in redshifts:
        model.set(z=z)
        mabs = np.random.normal(mag[0], mag[1])
        model.set_source_peakabsmag(mabs, 'bessellb', 'vega', cosmo=Planck15)
        amp.append(model.get('amplitude'))

    return {
        'amplitude': np.array(amp),
        'hostr_v': r_v * np.ones(len(redshifts)),
        'hostebv': np.random.exponential(ebv_rate, len(redshifts))
    }

transientprop = dict(lcmodel=model,
                    lcsimul_func=random_parameters,
                    lcsimul_prop=dict(mag=(-16.75, 0.98)))

z_max = 0.1
tr = simsurvey.get_transient_generator([0.0, z_max], ratekind='custom',
                                       ratefunc=lambda z: 1.5e-4,
                                       dec_range=[-30,90],
                                       mjd_range=[mjd_range[0] - model.maxtime()*(1.+z_max),
                                                  mjd_range[1] + model.mintime()*(1.+z_max)],
                                       transientprop=transientprop,
                                       sfd98_dir=sfd98_dir)

In [ ]:
instprop = {"desg":{"gain":1.,"zp":30,"zpsys":'ab'},
            "desr":{"gain":1.,"zp":30,"zpsys":'ab'}}

survey = simsurvey.SimulSurvey(generator=tr, plan=plan, instprop=instprop)
    
lcs = survey.get_lightcurves(
    # This cell may give an error because of the progress bar widget.
    # Just delete the following line to fix this.
    # progress_bar=True, notebook=True
)

Determining field IDs for all objects


Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"



Generating lightcurves


Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


In [6]:
lcs[0]

time,band,flux,fluxerr,zp,zpsys
float64,str4,float64,float64,int64,str2
57394.1014317,desg,3527.87201701,361.529360548,30,ab
57408.136021,desg,6539.62422549,1250.90454407,30,ab
57408.1364839,desg,3059.73157805,1250.90454025,30,ab
57408.1369469,desg,2836.0188506,1250.90453643,30,ab
